In [1]:
import pandas as pd

# Define column names (since iris.data typically has no headers)
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

# Read the CSV file
iris_data = pd.read_csv('iris.data', header=None, names=column_names)

# Display the first few rows
print(iris_data.head())

   sepal_length  sepal_width  petal_length  petal_width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


In [2]:
import cvxpy as cp
import numpy as np
import pandas as pd

def linear_least_squares_classifier(df, target_class):
    # Prepare data: Features (X) and binary labels (y)
    X = df.iloc[:, :-1].values  # All columns except the last
    y = np.where(df['class'] == target_class, 1, -1)  # 1 for target class, -1 otherwise
    
    n_features = X.shape[1]
    
    # Define optimization variables
    a = cp.Variable(n_features)  # Weight vector
    b = cp.Variable()           # Bias term
    
    # Define the least squares problem
    objective = cp.Minimize(cp.sum_squares(X @ a + b - y))
    problem = cp.Problem(objective)
    
    # Solve the problem
    problem.solve()
    
    return a.value, b.value

In [3]:
# Example usage:
# Load the Iris dataset (assuming iris_data is loaded as before)
iris_data = pd.read_csv('iris.data', header=None, names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'])

# Fit a linear classifier for 'Iris-setosa'
a, b = linear_least_squares_classifier(iris_data, 'Iris-setosa')

print("Weights (a):", a)
print("Bias (b):", b)

Weights (a): [ 0.1312861   0.48494601 -0.44552275 -0.12670283]
Bias (b): -0.7550609184556691


In [4]:

def train_test_split_pandas(df, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    
    # Shuffle the DataFrame
    df_shuffled = df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    # Split into train/validation
    n = len(df_shuffled)
    n_val = int(n * test_size)
    
    df_val = df_shuffled.iloc[:n_val]
    df_train = df_shuffled.iloc[n_val:]
    
    return df_train, df_val

def calculate_classification_error(y_true, y_pred):
    return np.mean(y_true != y_pred) * 100

In [19]:
# Load the Iris dataset
iris_data = pd.read_csv('iris.data', header=None, 
                        names=['sepal_length', 'sepal_width', 
                               'petal_length', 'petal_width', 'class'])

# Split into train/validation 
df_train, df_val = train_test_split_pandas(iris_data, test_size=1/3)

# Train linear classifier (using your CVXPY function)
a, b = linear_least_squares_classifier(df_train, target_class='Iris-setosa')

# Prepare validation data
X_val = df_val.iloc[:, :-1].values
y_val_true = np.where(df_val['class'] == 'Iris-setosa', 1, -1)

# Predict on validation set
y_val_pred = np.sign(X_val @ a + b)

# Calculate error
error = calculate_classification_error(y_val_true, y_val_pred)
print(f"Validation Error: {error:.2f}%")

Validation Error: 0.00%


In [29]:
def train_linear_multiclass_classifier(df):
    classes = df['class'].unique()  # List of unique classes
    classifiers = {}
    
    for target_class in classes:
        # Train a binary classifier for each class
        a, b = linear_least_squares_classifier(df, target_class)
        classifiers[target_class] = (a, b)
    
    return classifiers

In [33]:
def svm_hinge_loss_classifier(df, target_class):
   
    # Prepare data
    X = df.iloc[:, :-1].values  # Features
    y = np.where(df['class'] == target_class, 1, -1)  # Binary labels
    
    n_samples, n_features = X.shape
    
    # Define variables
    a = cp.Variable(n_features)  # Weight vector
    b = cp.Variable()            # Bias term
    
    # Hinge loss: sum(max(0, 1 - y_i*(a^T x_i + b)))
    hinge_loss = cp.sum(cp.pos(1 - cp.multiply(y, X @ a + b)))
    
    # Solve the problem (no regularization)
    problem = cp.Problem(cp.Minimize(hinge_loss))
    problem.solve()
    
    return a.value, b.value

In [31]:
def predict_multiclass(X, classifiers):
    scores = {}
    for class_name, (a, b) in classifiers.items():
        scores[class_name] = X @ a + b  # Decision score for each class
    
    # Select the class with the highest score
    return np.array([max(scores, key=lambda k: scores[k][i]) for i in range(len(X))])

In [35]:
def train_svm_multiclass_classifier(df):
    classes = df['class'].unique()  # List of unique classes
    classifiers = {}
    
    for target_class in classes:
        # Train a binary classifier for each class
        a, b = svm_hinge_loss_classifier(df, target_class)
        classifiers[target_class] = (a, b)
    
    return classifiers

In [36]:
# 1. Load and split the Iris dataset
iris_data = pd.read_csv('iris.data', header=None, 
                        names=['sepal_length', 'sepal_width', 
                               'petal_length', 'petal_width', 'class'])
df_train, df_val = train_test_split_pandas(iris_data, test_size=0.2, random_state=42)

# 2. Train the multi-class classifier
classifiers = train_linear_multiclass_classifier(df_train)

# 3. Predict on validation data
X_val = df_val.iloc[:, :-1].values
y_val_pred = predict_multiclass(X_val, classifiers)

# 4. Calculate error
y_val_true = df_val['class'].values
error = calculate_classification_error(y_val_true, y_val_pred)
print(f"Validation Error: {error:.2f}%")

Validation Error: 13.33%


In [37]:
# Load the Iris dataset
iris_data = pd.read_csv('iris.data', header=None, 
                        names=['sepal_length', 'sepal_width', 
                               'petal_length', 'petal_width', 'class'])

# Split into train/validation 
df_train, df_val = train_test_split_pandas(iris_data, test_size=1/3)

# Train linear classifier (using your CVXPY function)
a, b = svm_hinge_loss_classifier(df_train, target_class='Iris-setosa')

# Prepare validation data
X_val = df_val.iloc[:, :-1].values
y_val_true = np.where(df_val['class'] == 'Iris-setosa', 1, -1)

# Predict on validation set
y_val_pred = np.sign(X_val @ a + b)

# Calculate error
error = calculate_classification_error(y_val_true, y_val_pred)
print(f"Validation Error: {error:.2f}%")

Validation Error: 0.00%


In [39]:
# 1. Load and split the Iris dataset
iris_data = pd.read_csv('iris.data', header=None, 
                        names=['sepal_length', 'sepal_width', 
                               'petal_length', 'petal_width', 'class'])
df_train, df_val = train_test_split_pandas(iris_data, test_size=0.2, random_state=42)

# 2. Train the multi-class classifier
classifiers = train_svm_multiclass_classifier(df_train)

# 3. Predict on validation data
X_val = df_val.iloc[:, :-1].values
y_val_pred = predict_multiclass(X_val, classifiers)

# 4. Calculate error
y_val_true = df_val['class'].values
error = calculate_classification_error(y_val_true, y_val_pred)
print(f"Validation Error: {error:.2f}%")

Validation Error: 0.00%


In [40]:
def compute_confusion_matrix(classifiers, df_val):
    X_val = df_val.iloc[:, :-1].values
    y_true = df_val['class'].values
    y_pred = predict_multiclass(X_val, classifiers)
    
    classes = sorted(np.unique(y_true))
    cm = np.zeros((len(classes), len(classes)), dtype=int)
    
    for i, true_class in enumerate(classes):
        for j, pred_class in enumerate(classes):
            cm[i, j] = np.sum((y_true == true_class) & (y_pred == pred_class))
    
    return pd.DataFrame(cm, index=classes, columns=classes)

In [43]:
# Load data
iris_data = pd.read_csv('iris.data', header=None, 
                        names=['sepal_length', 'sepal_width', 
                               'petal_length', 'petal_width', 'class'])

# Split data
df_train, df_val = train_test_split_pandas(iris_data, test_size=1/3)

# Train multi-class SVM
classifiers = train_linear_multiclass_classifier(df_train)

# Compute confusion matrix
cm = compute_confusion_matrix(classifiers, df_val)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
                 Iris-setosa  Iris-versicolor  Iris-virginica
Iris-setosa               19                0               0
Iris-versicolor            0                7               8
Iris-virginica             0                2              14
